In [121]:
import psycopg2
import os
import pandas as pd
import ast
import re

from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()
pd.set_option('display.max_rows', 100000)
pd.set_option("max_colwidth", 500)
KEY_POSTGRES = os.getenv("KEY_POSTGRES")

In [64]:
DF_input = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\preparedInstitutionsTestDatabaseAccess.csv


In [65]:
df = pd.read_csv(DF_input)

In [66]:
s = df.iloc[2]["NER_BERT"]

In [67]:
df.iloc[2]["Institutions_man"]

"['Instituto de Investigação e Inovação em Saúde', 'University of Porto', 'University of British Columbia', 'BC Cancer', 'Institute of Molecular Pathology and Immunology']"

In [68]:
cleaned = re.sub(r"np\.float\d+\((.*?)\)", r"\1", s)
cleaned

"[{'entity_group': 'INS', 'score': 0.99988544, 'word': 'University of British Columbia', 'start': 32, 'end': 62}, {'entity_group': 'SUBC', 'score': 0.99957687, 'word': 'Vancouver', 'start': 64, 'end': 73}, {'entity_group': 'COU', 'score': 0.9991843, 'word': 'Canada', 'start': 75, 'end': 81}, {'entity_group': 'INS', 'score': 0.9996547, 'word': 'Hereditary Cancer', 'start': 112, 'end': 129}, {'entity_group': 'INS', 'score': 0.9165015, 'word': 'BC Cancer', 'start': 131, 'end': 140}, {'entity_group': 'INS', 'score': 0.9998889, 'word': 'University of British Columbia', 'start': 196, 'end': 226}, {'entity_group': 'SUBC', 'score': 0.99966955, 'word': 'Vancouver', 'start': 228, 'end': 237}, {'entity_group': 'COU', 'score': 0.9899244, 'word': 'Canada', 'start': 239, 'end': 245}, {'entity_group': 'INS', 'score': 0.9998265, 'word': 'Instituto de Investigação e Inovação em Saúde', 'start': 253, 'end': 298}, {'entity_group': 'INS', 'score': 0.9998478, 'word': 'Institute of Molecular Pathology and I

In [91]:
query = ast.literal_eval(cleaned)[6]["word"]
query

'Vancouver'

In [92]:
connection = psycopg2.connect(database="ror_institutions", user="postgres", password=KEY_POSTGRES, host="localhost", port=5432)
cursor = connection.cursor()
cursor.execute("SELECT DISTINCT ON (institution_id) name FROM public.institution_names WHERE name = %s;", (query,))
record = cursor.fetchall()
record

[]

In [53]:
ast.literal_eval(cleaned)[0]

{'entity_group': 'INS',
 'score': 0.99988544,
 'word': 'University of British Columbia',
 'start': 32,
 'end': 62}

In [106]:
df['Institutions_db'] = [[] for _ in range(len(df))]
df['Countries_db'] = [[] for _ in range(len(df))]

for row in tqdm(range(len(df))):
    cleaned = re.sub(r"np\.float\d+\((.*?)\)", r"\1", df.iloc[row]["NER_BERT"])
    for NER_result in ast.literal_eval(cleaned):
        if NER_result['entity_group'] == 'INS':
            query = NER_result['word']
            connection = psycopg2.connect(database="ror_institutions", user="postgres", password=KEY_POSTGRES, host="localhost", port=5432)
            cursor = connection.cursor()
            cursor.execute(f"SELECT * FROM institution_names WHERE name = '{query}';")
            record = cursor.fetchall()
            institutions_db = []
            for el in record:
                institutions_db.append(el[3])
            institutions_db = list(set(institutions_db))
            if len(institutions_db) == 1:
                df.iloc[row]['Institutions_db'].append(institutions_db)
                i = 0
                while i == 0:
                    for el in record:
                        if institutions_db[0] in el:
                            new_query = el[2]
                            connection = psycopg2.connect(database="ror_institutions", user="postgres", password=KEY_POSTGRES, host="localhost", port=5432)
                            cursor = connection.cursor()
                            cursor.execute(f"SELECT country_name FROM institution_locations WHERE country_name = '{new_query}';")
                            new_record = cursor.fetchall()
                            if len(new_record) > 0:
                                df.iloc[row]['Countries_db'].append(new_record[0][5])
                                i += 1

  2%|█▌                                                                              | 2/103 [02:20<1:58:08, 70.18s/it]


KeyboardInterrupt: 

# WITH CACHE

In [ ]:
# Create empty columns
df['Institutions_db'] = [[] for _ in range(len(df))]
df['Countries_db'] = [[] for _ in range(len(df))]

# Open ONE connection
connection = psycopg2.connect(
    database="ror_institutions",
    user="postgres",
    password=KEY_POSTGRES,
    host="localhost",
    port=5432
)
cursor = connection.cursor()

# Cache to avoid repeated DB hits
institution_cache = {}

for row in tqdm(range(len(df))):
    print(row)
    if pd.isna(df.at[row, "NER_BERT"]):    
        continue
        
    cleaned = re.sub(r"np\.float\d+\((.*?)\)", r"\1", df.at[row, "NER_BERT"])
    ner_results = ast.literal_eval(cleaned)
    
    for ner in ner_results:
        if ner.get('entity_group') != 'INS':
            continue
    
        query = ner.get('word')

        
        if '\'' in query:
            query = re.sub("\'", "\'\'", query)
    
        # ---- Cache lookup ----
        if query in institution_cache:
            institutions_db, countries_db = institution_cache[query]
        else:
            # Parameterized query
            cursor.execute(
                "SELECT * FROM institution_names WHERE name = %s;",
                (query,)
            )
            records = cursor.fetchall()
    
            institutions_db = list({el[1] for el in records})
            print(institutions_db)
            
            institutions_id = list({el[0] for el in records})
            print(institutions_id)
            
            countries_db = []
    
            if len(set(institutions_id)) == 1:
    
                # Find related country via institution_locations
                for el in records:
                    #if institution_id in el:
                    new_query = el[0]
    
                    cursor.execute(
                        "SELECT country_name FROM institution_locations WHERE institution_id = %s;",
                        (new_query,)
                    )
                    location_records = cursor.fetchall()
    
                    if location_records:
                        countries_db.append(location_records[0][0])
                        break

            
            institution_cache[query] = (institutions_db, countries_db)
    
        # Append results safely
        if institutions_db:
            df.at[row, 'Institutions_db'].append(institutions_db[0])
        if countries_db:
            df.at[row, 'Countries_db'].append(countries_db[0])
            
cursor.close()
connection.close()

# NO CACHE

In [179]:
# Create empty columns
df['Institutions_db'] = [[] for _ in range(len(df))]
df['Countries_db'] = [[] for _ in range(len(df))]

# Open ONE connection
connection = psycopg2.connect(
    database="ror_institutions",
    user="postgres",
    password=KEY_POSTGRES,
    host="localhost",
    port=5432
)
cursor = connection.cursor()

for row in tqdm(range(len(df))):
    print(row)
    if pd.isna(df.at[row, "NER_BERT"]):    
        continue
        
    cleaned = re.sub(r"np\.float\d+\((.*?)\)", r"\1", df.at[row, "NER_BERT"])
    ner_results = ast.literal_eval(cleaned)
    
    for ner in ner_results:
        if ner.get('entity_group') != 'INS':
            continue
    
        query = ner.get('word')

        
        if '\'' in query:
            query = re.sub("\'", "\'\'", query)
    
        # Parameterized query
        cursor.execute(
            "SELECT * FROM institution_names WHERE name = %s;",
            (query,)
        )
        records = cursor.fetchall()
    
        institutions_db = list({el[1] for el in records})
        print(institutions_db)
            
        institutions_id = list({el[0] for el in records})
        print(institutions_id)
            
        countries_db = []
    
        if len(set(institutions_id)) == 1:
    
            # Find related country via institution_locations
            for el in records:
                #if institution_id in el:
                new_query = el[0]
    
                cursor.execute(
                    "SELECT country_name FROM institution_locations WHERE institution_id = %s;",
                    (new_query,)
                )
                location_records = cursor.fetchall()
    
                if location_records:
                    countries_db.append(location_records[0][0])
                    break
    
        # Append results safely
        if institutions_db and institutions_db[0] not in df.at[row, 'Institutions_db']:
            df.at[row, 'Institutions_db'].append(institutions_db[0])
            if countries_db:
                df.at[row, 'Countries_db'].append(countries_db[0])
            
cursor.close()
connection.close()

  0%|                                                                                          | 0/103 [00:00<?, ?it/s]

0
1
2
['University of British Columbia']
['03rmrcq20']
[]
[]
[]
[]
['University of British Columbia']
['03rmrcq20']


  3%|██▍                                                                               | 3/103 [00:00<00:16,  6.17it/s]

[]
[]
[]
[]
['University of Porto']
['043pwc612']
[]
[]
['University of Porto']
['043pwc612']
3
[]
[]
['University of Cincinnati']
['01e3m7079']
[]
[]
[]
[]
['Emory University']
['03czfpz43']
[]
[]
[]
[]


  4%|███▏                                                                              | 4/103 [00:00<00:21,  4.58it/s]

...

[]
[]
[]
[]
[]
[]
91
[]
[]
['Boston University']
['05qwgg493']
92
93
94


 92%|██████████████████████████████████████████████████████████████████████████▋      | 95/103 [00:21<00:01,  6.38it/s]

['Georgetown University Medical Center']
['00hjz7x27']
95
['University of South Florida']
['032db5x82']
96
[]
[]
97
[]
[]


100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [00:21<00:00,  4.75it/s]

['Montefiore Medical Center']
['044ntvm43']
98
[]
[]
99
[]
[]
100
['Paul Scherrer Institute']
['03eh3y714']
101
['Wayne State University']
['01070mq45']
102


In [180]:
df[["NER_BERT", "Institutions_man", "Institutions_db"]]

,NER_BERT,Institutions_man,Institutions_db
0,[],NaN,[]
1,[],NaN,[]
2,"[{'entity_group': 'INS', 'score': np.float32(0.99988544), 'word': 'University of British Columbia', 'start': 32, 'end': 62}, {'entity_group': 'SUBC', 'score': np.float32(0.99957687), 'word': 'Vancouver', 'start': 64, 'end': 73}, {'entity_group': 'COU', 'score': np.float32(0.9991843), 'word': 'Canada', 'start': 75, 'end': 81}, {'entity_group': 'INS', 'score': np.float32(0.9996547), 'word': 'Hereditary Cancer', 'start': 112, 'end': 129}, {'entity_group': 'INS', 'score': np.float32(0.9165015), ...","['Instituto de Investigação e Inovação em Saúde', 'University of Porto', 'University of British Columbia', 'BC Cancer', 'Institute of Molecular Pathology and Immunology']","[University of British Columbia, University of Porto]"
3,"[{'entity_group': 'INS', 'score': np.float32(0.9998616), 'word': ""Cincinnati Children ' s Hospital Medical Center"", 'start': 84, 'end': 129}, {'entity_group': 'INS', 'score': np.float32(0.9997229), 'word': 'University of Cincinnati', 'start': 132, 'end': 156}, {'entity_group': 'SUBC', 'score': np.float32(0.9995741), 'word': 'Cincinnati', 'start': 178, 'end': 188}, {'entity_group': 'SUBC', 'score': np.float32(0.9996165), 'word': 'Ohio', 'start': 190, 'end': 194}, {'entity_group': 'INS', 'scor...","[""Cincinnati Children's Hospital Medical Center"", 'University of Cincinnati', ""Children's Hospital Los Angeles"", 'USC', 'Aflac Cancer and Blood Disorders Center', ""Children's Healthcare of Atlanta"", 'Emory University', 'GoBroad Healthcare Group', 'GoBroad Hospital']","[University of Cincinnati, Emory University]"
4,"[{'entity_group': 'INS', 'score': np.float32(0.92480797), 'word': 'Independent Contractor', 'start': 0, 'end': 22}, {'entity_group': 'SUBC', 'score': np.float32(0.9978091), 'word': 'Ashburn', 'start': 24, 'end': 31}, {'entity_group': 'COU', 'score': np.float32(0.9990839), 'word': 'USA', 'start': 37, 'end': 40}, {'entity_group': 'INS', 'score': np.float32(0.99686503), 'word': 'University of Pennsylvania', 'start': 104, 'end': 130}, {'entity_group': 'SUBC', 'score': np.float32(0.99908435), 'wo...","['University of Oklahoma Health Sciences Center', 'University at Buffalo', 'Battelle Memorial Institute', 'University of Memphis', 'Rutgers Institute for Nicotine & Tobacco Studies', 'Rutgers Biomedical and Health Sciences', 'University of Chicago', 'TSET Health Promotion Research Center', 'Stephenson Cancer Center', 'University of Pennsylvania']","[University of Pennsylvania, Stephenson Cancer Center, University of Oklahoma Health Sciences Center, University of Memphis, Battelle Memorial Institute, University of Chicago, Rutgers Biomedical and Health Sciences]"
5,"[{'entity_group': 'INS', 'score': np.float32(0.9999144), 'word': 'CONAHCYT', 'start': 0, 'end': 8}, {'entity_group': 'INS', 'score': np.float32(0.99088496), 'word': 'Center for Research on Population Health', 'start': 11, 'end': 51}, {'entity_group': 'INS', 'score': np.float32(0.9998376), 'word': 'National Institute of Public Health', 'start': 53, 'end': 88}, {'entity_group': 'SUBC', 'score': np.float32(0.9997936), 'word': 'Cuernavaca', 'start': 90, 'end': 100}, {'entity_group': 'COU', 'scor...","['Center for Research on Population Health', 'National Institute of Public Health', 'CONAHCYT', 'Institute for Research in Applied Mathematics and Systems', 'Universidad Nacional Autonoma de Mexico', 'Center for Research on Evaluation and Surveys', 'Harvard']","[CONAHCYT, National Institute of Public Health]"
6,"[{'entity_group': 'INS', 'score': np.float32(0.999654), 'word': 'Labex TOUCAN', 'start': 0, 'end': 12}, {'entity_group': 'INS', 'score': np.float32(0.8912306), 'word': 'ANR', 'start': 14, 'end': 17}, {'entity_group': 'SUBC', 'score': np.float32(0.9987576), 'word': 'Toulouse', 'start': 19, 'end': 27}, {'entity_group': 'COU', 'score': np.float32(0.99966335), 'word': 'France', 'start': 29, 'end': 35}, {'entity_group': 'INS', 'score': np.float32(0.99942905), 'word': 'Université 

In [182]:
df.iloc[4]["Authors"]

"[{'Name': 'Amy M Cohn', 'Affiliation': ['Department of Pediatrics, University of Oklahoma Health Sciences Center, Oklahoma City, OK, USA.', 'TSET Health Promotion Research Center, Stephenson Cancer Center, University of Oklahoma Health Sciences Center, Oklahoma City, OK, USA.']}, {'Name': 'Hoda Elmasry', 'Affiliation': ['Department of Pediatrics, University of Oklahoma Health Sciences Center, Oklahoma City, OK, USA.', 'Independent Contractor, Ashburn, VA, USA.']}, {'Name': 'Rebecca Ashare', 'Affiliation': ['Department of Psychology, University at Buffalo, Buffalo, NY, USA.']}, {'Name': 'Wallace Pickworth', 'Affiliation': ['Battelle Memorial Institute, Baltimore, MD, USA.']}, {'Name': 'James G Murphy', 'Affiliation': ['Department of Psychology, University of Memphis, Memphis, TN, USA.']}, {'Name': 'Andrea C Villanti', 'Affiliation': ['Rutgers Institute for Nicotine & Tobacco Studies, Rutgers Biomedical and Health Sciences, New Brunswick, NJ, USA.']}, {'Name': 'Donald Hedeker', 'Affilia

In [177]:
df[["Countries_man", "Countries_db"]]

,Countries_man,Countries_db
0,NaN,[]
1,NaN,[]
2,"['Portugal', 'Portugal', 'Canada', 'Canada', 'Portugal']","[Canada, Canada, Portugal, Portugal]"
3,"['United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'China', 'China']","[United States, United States]"
4,"['United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States', 'United States']","[United States, United States, United States, United States, United States, United States, United States, United States]"
5,"['Mexico', 'Mexico', 'Mexico', 'Mexico', 'Mexico', 'Mexico', 'United States']",[Mexico]
6,"['France', 'France', 'France', 'France', 'France', 'France', 'France']","[United States, France, United States, United States, France, France]"
7,['China'],"[China, China]"
8,"['Poland', 'Poland']","[Poland, Poland, Poland]"
9,"['Taiwan', 'Taiwan']","[Taiwan, Taiwan, Taiwan]"


In [ ]:
# Create empty columns
df['Institutions_db'] = [[] for _ in range(len(df))]
df['Countries_db'] = [[] for _ in range(len(df))]

# Open ONE connection
connection = psycopg2.connect(
    database="ror_institutions",
    user="postgres",
    password=KEY_POSTGRES,
    host="localhost",
    port=5432
)
cursor = connection.cursor()

# Cache to avoid repeated DB hits
institution_cache = {}

for row in tqdm(range(len(df))):
    if pd.isna(df.at[row, "NER_BERT"]):    
        continue
        
    cleaned = re.sub(r"np\.float\d+\((.*?)\)", r"\1", df.at[row, "NER_BERT"])
    ner_results = ast.literal_eval(cleaned)
    
    for ner in ner_results:
        if ner.get('entity_group') != 'INS':
            continue
    
        query = ner.get('word')

        
        if '\'' in query:
            query = re.sub("\'", "\'\'", query)
    
        # ---- Cache lookup ----
        if query in institution_cache:
            institutions_db, countries_db = institution_cache[query]
        else:
            # Parameterized query
            cursor.execute(
                "SELECT * FROM institution_names WHERE name = %s;",
                (query,)
            )
            records = cursor.fetchall()
    
            institutions_db = list({el[1] for el in records})
            institutions_id = list({el[0] for el in records})
            countries_db = []
    
            if len(set(institutions_db)) == 1:
    
                # Find related country via institution_locations
                for el in records:
                    if institution_id in el:
                        new_query = el[2]
    
                        cursor.execute(
                            "SELECT country_name FROM institution_locations WHERE country_name = %s;",
                            (new_query,)
                        )
                        location_records = cursor.fetchall()
    
                        if location_records:
                            countries_db.append(location_records[0][0])
                            break
    
            institution_cache[query] = (institutions_db, countries_db)
    
        # Append results safely
        if institutions_db:
            df.at[row, 'Institutions_db'].append(institutions_db)
        if countries_db:
            df.at[row, 'Countries_db'].append(countries_db)
            
cursor.close()
connection.close()